In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import scipy.special as poly

In [2]:
numsteps = 1000
burnin = 50
dof = 4
n = 100 + 1
totsteps = 25000
savesteps = 100
completedata = np.zeros(numsteps)
mmat = np.zeros((dof, dof))
rvec = np.zeros(dof)

x = np.zeros(savesteps)
t = np.zeros(savesteps)
h = (t[1]-t[0])/(n-1)
h12 = np.sqrt(h)
g = 1/2
coef = np.array([1., 2., -1., -0.5])

In [3]:
def frozendrift(x, coef):
    y = np.zeros(coef.size)
    for i in range(dof):
        H = poly.hermitenorm(i)
        y[i] = H(x)
    z = np.dot(y, coef)
    return z

In [4]:
def drift(x, theta):
    y = np.zeros(x.size)
    for i in range(x.size):
        for j in range(dof):
            H = poly.hermitenorm(j)
            y[i] = y[i] + theta[j] * H(x[i])

In [ ]:
traj = 1.0
j = 0
x[0] = traj
t[0] = 0.0
for i in range(1, totsteps):
    traj = traj + frozendrift(traj, coef) * h + g * h12 * np.random.rand(1)
    print(traj)
    if (i % (totsteps/savesteps) == 0):
        x[j] = traj
        t[j] = i*h
        j = j + 1

[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.]
[ 1.]
[ 1.  1.  0. -2.

In [6]:
def brownianbridge(xin, tin, h):
    tvec = tin[0] + (1+np.arange(n))*h
    h12 = np.sqrt(h)
    wincs = np.random.normal(scale=h12, size=n)
    w = np.cumsum(wincs)
    bridge = xin[0] + w - ((tvec - tin[0])/(tin[1]-tin[0]))*(w[n-1] + xin[0] - xin[1])
    return tvec, bridge

In [7]:
t[0]
# tout, bout = brownianbridge(x,t,h)
# tout = np.concatenate((np.zeros(1),tout))
# bout = np.concatenate((x[[0]],bout))
# plt.plot(tout,bout)

0.0

In [8]:
theta = np.array([10., 1., 2., 0.25])

In [9]:
# define drift function
# def drift(x, theta):
#     f = theta[0]*np.power(theta[1] + np.power(x,theta[2]),theta[3])
#     return f

In [10]:
# girsanov LOG likelihood
def girsanov(path, dt, theta):
    b = drift(path, theta)
    int1 = np.dot(b[:-1], np.diff(path))
    b2 = np.square(b)
    int2 = np.sum(0.5*(b2[1:] + b2[:-1]))*dt
    r = int1 - 0.5*int2
    return r

In [11]:
def mypoly(x):
    y = np.zeros((x.size, dof))
    for i in range(x.size):
        for j in range(dof):
            y[i,j] = poly.hermitenorm(j, x[i])
    return y

In [12]:
done = 0
mytol = np.power(10, -3)
numiter = 0
while (done == 0):
    numiter = numiter + 1
#     print(numiter)
    tout, xcur = brownianbridge(x,t,h)
    oldlik = girsanov(path=xcur, dt=h, theta=theta)

    for jj in range(numsteps + burnin):
        _, prop = brownianbridge(x,t,h)
        proplik = girsanov(path=prop, dt=h, theta=theta)
        rho = np.exp(proplik - oldlik)
        if (rho > np.random.uniform()):
            xcur = prop
            oldlik = proplik
        # ignore samples during burnin phase
        if (jj > burnin):
            pp = mypoly(xcur[:(-1)])
            mmat = mmat + h * np.matmul(pp.T, pp) / numsteps
            rvec = rvec + np.matmul((np.diff(xcur)).T, pp) / numsteps

    newtheta = np.linalg.solve(mmat, rvec)
    check = np.sum(np.abs(newtheta - theta))
    if (check < mytol):
        print("finished!")
        print(check)
        done = 1
    theta = newtheta
    print(check)
#     print(theta)

ValueError: scale <= 0